# The scientist's friends: NumPy, matplotlib and IPython

Python is quite popular in the astronomy community, and many tools are based on it (e.g. CASA and PyRAF). The KAT-7 and MeerKAT telescopes are controlled and monitored via Python and it is also used to reduce the recorded data, so it is important for us! A powerful combination for scientific data analysis is [*NumPy*](http://www.numpy.org) for arrays and basic linear algebra, [*SciPy*](http://www.scipy.org) for fancier algorithms, [*matplotlib*](http://matplotlib.org) for plotting and [*IPython*](http://ipython.org) as an interactive shell that binds the lot together. We start with a very quick tutorial on these.

The next cell loads the plotting and science analysis packages

In [ ]:
%pylab inline

Our tutorials are presented as IPython [notebooks](http://ipython.org/notebook.html) which allow you to run Python code within the tutorial page. A *code cell* looks like this:

Every time you encounter one, click on it to put your cursor inside it and hit Ctrl-Enter to execute the cell. The output of the cell will then appear below it. All the cells know of each other and changing a value inside one cell immediately modifies the variable in all other cells where it is found.

- Create a simple NumPy array of consecutive numbers and then index and slice it in various ways:

In [ ]:
n = arange(10)

In [ ]:
n

In [ ]:
n[0]

In [ ]:
n[-1]

In [ ]:
n[2:3]

In [ ]:
n[2:5]

In [ ]:
n[:5]

In [ ]:
n[5:]

- Now try your hand at generating samples from a sinusoidal function and plotting it. As you can see below, cells may contain multiple lines of code:

In [ ]:
t = arange(100)
x = cos(2 * pi * t / 40)
plot(t, x)

If you need any help on these functions, IPython provides two indispensable tools: *tab completion* and *object inspection*. Tab completion finds the names of functions, variables or anything else accessible within IPython that starts with the letters you have typed, by hitting the Tab key. For example, to find all entities starting with the letters *pl*, type ``pl`` and hit Tab:

In [ ]:
pl

This is very useful in case you have forgotten the exact name of a function (or even to explore what is available!).

Objection inspection prints out help related to an entity, by typing its name followed by a question mark (``?``):

In [ ]:
plot?

- Now load the data to be used for the rest of the tutorial into your workspace:

In [ ]:
data = np.load('single_dish_tutorial.npz')

We have loaded the data from a zipped set of numpy arrays on disk. The contents of the zipped set may be inspected by:


In [ ]:
data.files

# The radiometer

The most basic operation of a single-dish radio telescope is to act as a *radiometer*, which measures the received radio-frequency power from various regions of the sky and as a function of time. If this operation is done simultaneously in multiple frequency bands, the instrument is called a *spectrometer*.

Measuring power
---------------

A typical recording of voltage data produced by a radio telescope pointed at a celestial source (so-called *Gaussian* noise) looks like this:

In [ ]:
x = randn(500)
plot(x)

The radiometer estimates the power of this voltage signal by squaring and averaging the sample values. This process is repeated on long runs of consecutive samples (up to several seconds in length), producing a time series of power measurements. This time series is the starting point of our analysis.

Noise diode calibration
-----------------------

First off, we have to calibrate the units of the data. The power values produced by the KAT telescope are 16-bit numbers which range from -32768 to 32767. These values are uncalibrated, with a unit that is traditionally called *counts*. We need to convert this to a physical unit of power. The standard unit is the watt, but celestial radio sources are so weak that it is not very useful. The power measurement can be expressed as a *temperature* in kelvin (K) instead, through the relation

\begin{equation}
  P = \text{k}TB, \quad \text{[W]}
\end{equation}

where $P$ is the power in watts, $\text{k} = 1.3806504 \times 10^{-23}$ is Boltzmann's constant, $T$ is the temperature in kelvin and $B$ is the bandwidth in Hz. The advantage of temperature is that it is independent of bandwidth, while the power of a noise signal increases with increasing bandwidth.

The first step in calibration is to figure out the *receiver (or electronic) gain* which exists between the feed horn of the antenna and the voltage output. This is done by firing a noise diode, which is a calibrated noise source with a known and stable temperature, at the input of the feed horn and observing the increase in measured power. This allows us to determine the receiver gain in counts per kelvin.

- Now plot the time series that contains a noise diode switching on and off:

In [ ]:
power = data['nd_power']
plot(power)

- Given that the noise diode temperature was 3.7 K, estimate the receiver gain in counts/K. Use the code cell below as your calculator!

- Use this gain to estimate the temperature measured while the noise diode was off. This will provide an indication of the *system temperature* or $T_{\text{sys}}$, which is an important measure of the quality of the instrument. A lower system temperature allows the telescope to detect  fainter objects.

Linear scan across source
=========================

We know the positions of most strong radio sources very accurately, thanks to decades of observations. In this experiment, we move the dish across the position of a source and observe the measured power. The power increases as we move onto the source and decreases again as we move off it. This bump in power means we detected the source! Radio astronomy is full of bumps and blobs...

- Plot the scan power as a function of azimuth angle (in degrees) as below:

In [ ]:
az = data['scan_az']
power = data['scan_power']
plot(az, power)
xlabel('Azimuth angle (degrees)')
ylabel('Power (counts)')

What we are actually seeing is a convolution of the antenna *beam pattern* and the structure of the source. For a point source like the one shown, the shape of the bump is purely due to the beam pattern. This allows us to measure the properties of this pattern.

Calibration
-----------

The standard flux unit in radio astronomy is the *jansky (Jy)*, where $1\,\text{Jy} = 10^{-26}\,\text{W}\text{m}^{-2}\text{Hz}^{-1}$, used to quantify the *flux density* of the radio source. We therefore need to calibrate our scan from raw counts to temperatures in kelvin, all the way to flux densities in Jy. The conversion from temperature to flux density involves the gain introduced by the collecting area of the dish itself, referred to as *antenna gain*, which is given by

\begin{equation}
  G = \frac{\eta A}{2 \text{k}} \times 10^{-26} \approx \frac{\eta A}{2761}, \quad \text{[K / Jy]}
\end{equation}

where $\eta$ is the efficiency of the dish (less than 1!), $A$ is the dish area in square metres and $\text{k}$ is Boltzmann's constant.

- Estimate the antenna gain for a KAT-7 dish with a diameter of 12 metres and an efficiency of 0.5 (50%).

- Combined with the receiver gain in the previous section, use this to calibrate the scan power to be in Jy and make a new plot.

Parameter extraction
--------------------

- Estimate the baseline level of the scan (the lowest part), in Jy. This is also known as the *system equivalent flux density*, or SEFD, which is equivalent to the system temperature.

- Estimate the flux density of the source (in Jy) as the height of the peak of the scan above the baseline level.

- Estimate the *beamwidth* in degrees, which the full width of the beam pattern at half of its maximum value, also known as the FWHM. Zoom into the plot by issuing `axis`, `xlim` or `ylim` commands in order to help you read off the width.

# Raster scan image

Single-dish images are built up pixel by pixel by physically scanning the dish across a patch of sky in a grid or *raster* pattern (incidentally the same way your old-fashioned CRT TV builds up its images). These scans are then assembled into the final image. If the raster scan takes a long time or extends over a large region of sky, we have to convert the spherical antenna coordinates (azimuth angle and elevation angle, or right ascension and declination) to a projected plane centred on the target.

In the dataset you will find a raster scan of [Centaurus A](http://en.wikipedia.org/wiki/Centaurus_A), an impressive radio galaxy in the southern sky and one of the brightest celestial radio sources. The data was recorded using one of the KAT-7 dishes, in a frequency band centered at 1836 MHz and with a bandwidth of 222 MHz. The right ascension and declination coordinates have already been converted to a projected plane, and all that is left to do is to plot the image.

- Load the data and plot the individual pointings (in degrees) that make up the raster scan. Also plot the measured power as a function of time, and compare this with the scan from the previous task.

In [ ]:
ra = data['raster_ra']
dec = data['raster_dec']
power = data['raster_power']
plot(ra, dec, '.')
axis('image')
xlabel('Right Ascension (degrees)')
ylabel('Declination (degrees)')

In [ ]:
plot(power)
xlabel('Sample index')
ylabel('Flux density (Jy)')

The power plot shows the data as collected over time by the telescope while performing the raster scan over the radio source. In order to make an image, the data needs to be reorganized.

The first step to an image is to *regrid* the measurements onto a regular rectangular (2-dimensional) grid, where each element of the grid corresponds to a pixel. Matplotlib provides a function for this, called ``griddata``.

- Read the help on this function, select a pixel size in degrees (which determines the size of your image) and do the following (ignore the warning message about dateutil):

In [ ]:
pixel_size = 0.05
# Use the full (ra, dec) range for the plots...
grid_ra = np.arange(ra.min(), ra.max(), pixel_size)
grid_dec = np.arange(dec.min(), dec.max(), pixel_size)
# Or pick your own range to zoom in on the plots
#grid_ra = np.arange(200, 203, pixel_size)
#grid_dec = np.arange(-46, -41, pixel_size)
grid_power = griddata(ra, dec, power, grid_ra, grid_dec, interp='linear')

There are two popular ways to plot such an image: as a contour plot or a bitmap image. Here are some examples of how to do this:

In [ ]:
contour(grid_ra, grid_dec, grid_power, 50, cmap=cm.jet)
axis('image')
xlabel('Right Ascension (degrees)')
ylabel('Declination (degrees)')

In [ ]:
extent=[grid_ra.min(), grid_ra.max(), grid_dec.min(), grid_dec.max()]
imshow(grid_power, extent=extent, origin='lower', cmap=cm.gray)
xlabel('Right Ascension (degrees)')
ylabel('Declination (degrees)')

Feel free to modify the above commands. You can change the color scheme of the bitmap image -- for more information, read the ``colormaps?`` help.

The image does not show a lot of detail, compared to say [this image](http://www.cfa.harvard.edu/news/2010/su201049_images.html) of the radio lobes. How would you increase the resolution of the image?